In [ ]:
!pip install faiss-cpu sentence-transformers langchain transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 39.2 MB/s eta 0:00:00


In [ ]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [ ]:
from langchain.schema import Document

# Read the text file and split it into individual lines
with open("/content/data.txt", "r", encoding="utf-8") as file:
    texts = file.readlines()

# Optionally, clean up the lines by removing extra whitespace or newlines
texts = [line.strip() for line in texts if line.strip()]

# Convert each line into a Document object
documents = [Document(page_content=text) for text in texts]

# Print the documents to verify
for doc in documents:
    print(doc)


page_content='Artificial Intelligence (AI) is one of the most transformative technologies of the 21st century. It refers to the simulation of human intelligence in machines that are programmed to think, learn, and act autonomously. AI applications can be found across industries, from healthcare and education to entertainment and transportation.'
page_content='In healthcare, AI is revolutionizing diagnostics by analyzing medical images and predicting diseases with incredible accuracy. AI-powered chatbots assist patients by answering their health-related questions, and robotic surgery systems enhance precision during operations. Similarly, in the education sector, AI enables personalized learning experiences through adaptive learning platforms, which adjust the curriculum based on a student's individual progress.'
page_content='The entertainment industry benefits from AI through recommendation algorithms used by platforms like Netflix and Spotify, which analyze user preferences to sugges

In [ ]:
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.llms import HuggingFacePipeline
from langchain.schema import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline


# Initialize the embeddings model
embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

# Create a FAISS vectorstore
vectorstore = FAISS.from_documents(documents, embeddings)

# Use vectorstore for retrieval tasks
retriever = vectorstore.as_retriever()

# Create a better prompt template
prompt_template = """
Question: {question}

Context: {context}

"""

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

# Setup language model with better parameters
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

llm_pipeline = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=100,  # Reduced to avoid repetition

    truncation=True,
    pad_token_id=tokenizer.eos_token_id,
    do_sample=False,
    no_repeat_ngram_size=2  # Prevent repetition of phrases
)

llm = HuggingFacePipeline(pipeline=llm_pipeline)

# Create the QA chain with the custom prompt
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)



<ipython-input-5-d7de99df3227>:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.wa

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu
<ipython-input-5-d7de99df3227>:49: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=llm_pipeline)


In [ ]:
import time

query = "What is Artificial Intellegence?"
start_time = time.time()
response = llm_pipeline(query)  # Direct LLM query without retrieval
end_time = time.time()

baseline_time = end_time - start_time
print("Baseline Time (seconds):", baseline_time)
print("Response:", response[0]['generated_text'])

Baseline Time (seconds): 0.42293381690979004
Response: artificial intelligence


In [ ]:
import time

question = "What is Artificial Intellegence?"
start_time = time.time()
response = qa_chain({"query": question})
end_time = time.time()

response_time = end_time - start_time
print("Response Time (seconds):", response_time)
print("Response:", response['result'])

Response Time (seconds): 2.9535000324249268
Response: AI


In [ ]:
import time

# Define the queries to test the QA system
queries = [
    "What is Artificial Intelligence?",
    "How is AI used in healthcare?",
    "What are the ethical concerns of AI?",
    "How does Netflix use AI?",
    "What is the role of AI in transportation?",
    "What is the future impact of AI on society?"
]

# Initialize total_time to 0
total_time = 0

# Measure the response time for each query
for query in queries:
    start_time = time.time()  # Start timer
    response = qa_chain({"query": query})  # Query the RAG system
    total_time += time.time() - start_time  # Add elapsed time

# Calculate the average response time
Y = total_time / len(queries)
print("Average Response Time (seconds):", Y)

# Optionally, print the results for each query
for query in queries:
    print(f"Query: {query}")
    response = qa_chain({"query": query})
    print(f"Answer: {response['result']}\n")


Average Response Time (seconds): 2.9120305379231772
Query: What is Artificial Intelligence?
Answer: simulation of human intelligence in machines that are programmed to think, learn, and act autonomously

Query: How is AI used in healthcare?
Answer: revolutionizing diagnostics by analyzing medical images and predicting diseases with incredible accuracy

Query: What are the ethical concerns of AI?
Answer: job displacement, data privacy, and biased algorithms

Query: How does Netflix use AI?
Answer: recommendation algorithms

Query: What is the role of AI in transportation?
Answer: leveraging AI with autonomous vehicles

Query: What is the future impact of AI on society?
Answer: growth



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import time

# Example documents as plain text
plain_texts = [doc.page_content for doc in documents]

# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(plain_texts)

# Query embedding (brute force with TF-IDF)
query = "Artificial Intelligence"
start_time = time.time()
query_vec = vectorizer.transform([query])
similarities = (tfidf_matrix * query_vec.T).toarray()  # Dot product
end_time = time.time()

baseline_time = end_time - start_time
print("Baseline Time (seconds):", baseline_time)


Baseline Time (seconds): 0.0031549930572509766


In [ ]:
import time

# Simulate keyword search over the documents
def keyword_search(query, documents):
    return [doc for doc in documents if query.lower() in doc.page_content.lower()]

# Measure time for keyword search
query = "machine learning"
start_time = time.time()
results = keyword_search(query, documents)
end_time = time.time()

baseline_time = end_time - start_time
print("Baseline Time (seconds):", baseline_time)


Baseline Time (seconds): 0.00011467933654785156


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import time

# Setup models
gpt2_model_name = "gpt2"


gpt2_tokenizer = AutoTokenizer.from_pretrained(gpt2_model_name)
gpt2_model = AutoModelForCausalLM.from_pretrained(gpt2_model_name)



# Create pipelines
gpt2_pipeline = pipeline("text-generation", model=gpt2_model, tokenizer=gpt2_tokenizer)


# Define test queries
queries = [
    "What is Artificial Intelligence?",
    "How is AI used in healthcare?",
    "What are the ethical concerns of AI?",
    "How does Netflix use AI?",
    "What is the role of AI in transportation?",
    "What is the future impact of AI on society?"
]

# Measure response time and output for GPT-2 and Flan-T5
gpt2_times = []


for query in queries:
    start_time = time.time()
    gpt2_response = gpt2_pipeline(query)
    gpt2_times.append(time.time() - start_time)



# Calculate average response times
gpt2_avg_time = sum(gpt2_times) / len(gpt2_times)


print(f"GPT-2 Average Response Time: {gpt2_avg_time:.4f} seconds")

# Output for comparison
print(f"\nGPT-2 Responses:")
for query, response in zip(queries, gpt2_response):
    print(f"{query}: {response['generated_text']}")




Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


GPT-2 Average Response Time: 2.7612 seconds

GPT-2 Responses:
What is Artificial Intelligence?: What is the future impact of AI on society?

The challenge is so great we don't even need a PhD. We just need a computer.


What is AI? How do we make better AI?

When I first heard
